# 1. Improt  and Install Dependencies

In [1]:
# !pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import  mediapipe  as mp

# 2. Keypoint using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic Model
mp_drawing  = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def  mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #converting color
    image.flags.writeable = False # Image is no longer writeable
    results = model.process(image) #make prediction
    image.flags.writeable = True # Image is now writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #converting color
    return image,results

# This is optional now,The one being used is the draw_styled_landmarks.

In [5]:
def draw_styled_landmarks(image, result):
    mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(250,250,250), thickness = 2, circle_radius = 2),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius = 1)) # Draw pose conections
    mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS) # Draw left Hand conections
    mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) # Draw Right Hand conection.

# Optional {For testing the camera and detection mechanisim}

In [6]:
# cap  = cv2.VideoCapture(1)
# # Accessing mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence = 0.5) as holistic:
#     while cap.isOpened():
        
#         # Read feed
#         ret,frame = cap.read()

#         # Make detection
#         image, results = mediapipe_detection(frame,holistic)
        
        
#         # Draw  landmarks
#         draw_styled_landmarks(image,results)
        

#         cv2.imshow('RSL Feed', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

# 3. Extract Keypoint Values

In [7]:

# With out Face points  multiple left and hand cordinates to increase accuracy
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    # magnifiying the hands value for better accuracy
    return np.concatenate([pose, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh])

# 4. Setup Folders for Collection 

In [8]:
# Output will be  arrray containing numerical values for the extracted keypoints
# Path to outr external data
DATA_PATH = os.path.join('..', 'signlang-kinyarwanda/Data')  # Adjust path as needed


# Check if the folder exists
if os.path.isdir(DATA_PATH):
    print("Folder exists:", DATA_PATH)
else:
    print("Folder NOT found:", DATA_PATH)



Folder exists: ..\signlang-kinyarwanda/Data


In [9]:
# Get all action folders from the external dataset
action_folders = [f for f in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, f))]
actions = np.array(sorted(action_folders))  # Sort for consistency

print(f"Found {len(actions)} actions in the dataset:")
print(actions)


Found 15 actions in the dataset:
['---' 'akarere' 'akazi' 'amakuru' 'bayi' 'bibi' 'igihugu' 'neza'
 'nyarugenge' 'oya' 'papa' 'umujyi wa kigali' 'umurenge' 'urakoze' 'yego']


In [10]:

# Auto-detect number of sequences and frames
sample_action = actions[0]
sample_path = os.path.join(DATA_PATH, sample_action)
sequences_in_action = [d for d in os.listdir(sample_path) if os.path.isdir(os.path.join(sample_path, d))]
no_sequences = len(sequences_in_action)

# Check frames in first sequence
first_sequence_path = os.path.join(sample_path, sequences_in_action[0])
npy_files = [f for f in os.listdir(first_sequence_path) if f.endswith('.npy')]
sequence_length = len(npy_files)

print(f"\nDataset Info:")
print(f"- Number of sequences per action: {no_sequences}")
print(f"- Frames per sequence: {sequence_length}")

print(" Now we have a total of {} videos from all actions.".format(len(actions)*no_sequences))


Dataset Info:
- Number of sequences per action: 40
- Frames per sequence: 30
 Now we have a total of 600 videos from all actions.


In [11]:
# Inspect the data structure
import glob

print("=" * 60)
print("DATA STRUCTURE INSPECTION")
print("=" * 60)

for action in actions[:3]:  # Check first 3 actions
    action_path = os.path.join(DATA_PATH, action)
    sequences = sorted([d for d in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, d))])
    
    print(f"\nAction: {action}")
    print(f"  Total sequences: {len(sequences)}")
    
    # Check first sequence
    if sequences:
        first_seq_path = os.path.join(action_path, sequences[0])
        npy_files = sorted([f for f in os.listdir(first_seq_path) if f.endswith('.npy')])
        
        print(f"  First sequence: {sequences[0]}")
        print(f"    Number of .npy files: {len(npy_files)}")
        
        # Load one file to check shape
        if npy_files:
            sample_file = os.path.join(first_seq_path, npy_files[0])
            sample_data = np.load(sample_file)
            print(f"    Sample file shape: {sample_data.shape}")
            print(f"    Feature dimensions: {sample_data.shape[0] if len(sample_data.shape) == 1 else sample_data.shape}")

print("\n" + "=" * 60)

DATA STRUCTURE INSPECTION

Action: ---
  Total sequences: 40
  First sequence: 1
    Number of .npy files: 30
    Sample file shape: (1518,)
    Feature dimensions: 1518

Action: akarere
  Total sequences: 40
  First sequence: 1
    Number of .npy files: 30
    Sample file shape: (1518,)
    Feature dimensions: 1518

Action: akazi
  Total sequences: 40
  First sequence: 1
    Number of .npy files: 30
    Sample file shape: (1518,)
    Feature dimensions: 1518



# 5. Collect Keypoint Values for Training and Testing

# SKIPPED
**Note:** Data collection is skipped because we're using pre-existing `.npy` files from the external dataset.

# 6. Preprocess External Data and Create Labels and Features


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:


# Create label map
label_map = {label: num for num, label in enumerate(actions)}
print("Label Mapping:")
for label, num in label_map.items():
    print(f"  {num}: {label}")

# Load sequences and labels
sequences, labels = [], []
missing_data = []

print("\nLoading data...")
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    
    # Get all sequence folders
    sequence_folders = sorted([d for d in os.listdir(action_path) 
                               if os.path.isdir(os.path.join(action_path, d))])
    
    for sequence_folder in sequence_folders:
        sequence_path = os.path.join(action_path, sequence_folder)
        window = []
        
        try:
            # Get all .npy files in sequence
            npy_files = sorted([f for f in os.listdir(sequence_path) if f.endswith('.npy')])
            
            if len(npy_files) != sequence_length:
                missing_data.append(f"{action}/{sequence_folder} - Expected {sequence_length}, found {len(npy_files)}")
                continue
            
            # Load each frame
            for npy_file in npy_files:
                npy_path = os.path.join(sequence_path, npy_file)
                res = np.load(npy_path)
                window.append(res)
            
            sequences.append(window)
            labels.append(label_map[action])
            
        except Exception as e:
            missing_data.append(f"{action}/{sequence_folder} - Error: {str(e)}")
            continue

print(f"\nLoaded {len(sequences)} sequences successfully")

if missing_data:
    print(f"\nWarning: {len(missing_data)} sequences had issues:")
    for item in missing_data[:10]:  # Show first 10
        print(f"  - {item}")
    if len(missing_data) > 10:
        print(f"  ... and {len(missing_data) - 10} more")

Label Mapping:
  0: ---
  1: akarere
  2: akazi
  3: amakuru
  4: bayi
  5: bibi
  6: igihugu
  7: neza
  8: nyarugenge
  9: oya
  10: papa
  11: umujyi wa kigali
  12: umurenge
  13: urakoze
  14: yego

Loading data...


In [ ]:
# Verify loaded data
print("\nData Shapes:")
print(f"Sequences array shape: {np.array(sequences).shape}")
print(f"Labels array shape: {np.array(labels).shape}")

# Check feature dimensions
if len(sequences) > 0:
    print(f"\nFeature dimensions per frame: {len(sequences[0][0])}")
    print(f"Expected: 1392 (Phase 3 with magnified hands)")
    
    # Distribution of labels
    unique_labels, counts = np.unique(labels, return_counts=True)
    print("\nLabel Distribution:")
    for label_idx, count in zip(unique_labels, counts):
        action_name = actions[label_idx]
        print(f"  {action_name}: {count} sequences")


Data Shapes:
Sequences array shape: (351, 30, 1518)
Labels array shape: (351,)

Feature dimensions per frame: 1518
Expected: 1392 (Phase 3 with magnified hands)

Label Distribution:
  ---: 40 sequences
  akarere: 40 sequences
  akazi: 40 sequences
  amakuru: 40 sequences
  bayi: 40 sequences
  bibi: 40 sequences
  igihugu: 40 sequences
  neza: 40 sequences
  yego: 31 sequences


# 7. Convert to arrays and split

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

print(f"\nFinal shapes:")
print(f"X (features): {X.shape}")  # Should be (num_sequences, 30, feature_dim)
print(f"y (labels): {y.shape}")    # Should be (num_sequences, num_actions)

# Get actual feature dimensions
actual_feature_dim = X.shape[2]
print(f"\nActual feature dimensions: {actual_feature_dim}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=labels)

print(f"\nTrain-Test Split:")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")


Final shapes:
X (features): (351, 30, 1518)
y (labels): (351, 15)

Actual feature dimensions: 1518

Train-Test Split:
Training samples: 245
Testing samples: 106


# 8. Build Model with Correct Input Shape


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import regularizers

# Setup callbacks
log_dir = os.path.join('Logs_External_Data')
tb_callback = TensorBoard(log_dir=log_dir)
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Optimizer
opt = Adam(learning_rate=0.00001)

# Model architecture - UPDATE INPUT SHAPE
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu',
               kernel_regularizer=regularizers.l2(0.01), 
               input_shape=(sequence_length, actual_feature_dim)))  # Use actual dimensions
model.add(LSTM(128, return_sequences=True, activation='relu', 
               kernel_regularizer=regularizers.l2(0.01)))
model.add(LSTM(64, return_sequences=False, activation='relu', 
               kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(len(actions), activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

# Compile
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Summary
model.summary()

C:\Users\STUDENT\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │       405,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 560,207 (2.14 MB)

 Trainable params: 560,207 (2.14 MB)

 Non-trainable params: 0 (0.00 B)

# 9. Train the model

In [ ]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=200,
    validation_data=(X_test, y_test),  # Use test set for validation
    callbacks=[tb_callback, early_stop],
    batch_size=32,
    verbose=1
)


Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - categorical_accuracy: 0.9668 - loss: 7.2723 - val_categorical_accuracy: 0.8491 - val_loss: 7.8648
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - categorical_accuracy: 0.9725 - loss: 7.2653 - val_categorical_accuracy: 0.8491 - val_loss: 7.8637
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - categorical_accuracy: 0.9803 - loss: 7.2394 - val_categorical_accuracy: 0.8491 - val_loss: 7.9038
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - categorical_accuracy: 0.9733 - loss: 7.2451 - val_categorical_accuracy: 0.8491 - val_loss: 7.8765
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - categorical_accuracy: 0.9763 - loss: 7.2511 - val_categorical_accuracy: 0.8491 - val_loss: 7.8943
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - categorical_accuracy: 0.9734 - loss: 7.2389 - val_categorical_accuracy: 0.8491 - val_loss: 7.9136
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - categorical_accuracy: 0.9826 - loss: 7.17

## Plot training history

In [ ]:


plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['categorical_accuracy'], label='Train Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Mak